Hometask Question 1 lesson 3 

In [156]:
import pandas as pd
import numpy as np
import re as re

Перевіряємо версію встановленої бібліотеки

In [157]:
pd.__version__

'2.2.2'

In [158]:
energy = pd.read_excel(
    "Energy Indicators.xls",
    sheet_name="Energy",
    skiprows= 18, #Пропускаємо перші 18 рядків
    skipfooter= 38, #Пропускаємо останні 38 рядків
    usecols= "C:F", #Вибираємо лише потрібні стовпці
    header= None
)
print(energy)

                             2     3    4           5
0                  Afghanistan   321   10   78.669280
1                      Albania   102   35  100.000000
2                      Algeria  1959   51    0.551010
3               American Samoa   ...  ...    0.641026
4                      Andorra     9  121   88.695650
..                         ...   ...  ...         ...
222                   Viet Nam  2554   28   45.321520
223  Wallis and Futuna Islands     0   26    0.000000
224                      Yemen   344   13    0.000000
225                     Zambia   400   26   99.714670
226                   Zimbabwe   480   32   52.536120

[227 rows x 4 columns]


Перейменовуємо стовпці

In [159]:
energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

Заміняємо ... на Nan

In [160]:
energy.replace('...', np.nan, inplace=True)

C:\Users\userk\AppData\Local\Temp\ipykernel_11464\801257677.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  energy.replace('...', np.nan, inplace=True)


Конвертуємо петаджоулі в гігаджоулі

In [161]:
energy['Energy Supply'] *= 1000000

Перейменовуємо країни

In [162]:
renamed_country = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}

Очищаємо назви країн від цифр й дужок

In [163]:
def clean_country(name):
    name = re.sub(r'\d+', '', name)  # Видалити числа
    name = re.sub(r'\s*\(.*\)', '', name)  # Видалити текст у дужках
    return name.strip()

energy['Country'] = energy['Country'].apply(clean_country)
energy['Country'] = energy['Country'].replace(renamed_country)

print(energy)

                       Country  Energy Supply  Energy Supply per Capita  \
0                  Afghanistan   3.210000e+08                      10.0   
1                      Albania   1.020000e+08                      35.0   
2                      Algeria   1.959000e+09                      51.0   
3               American Samoa            NaN                       NaN   
4                      Andorra   9.000000e+06                     121.0   
..                         ...            ...                       ...   
222                   Viet Nam   2.554000e+09                      28.0   
223  Wallis and Futuna Islands   0.000000e+00                      26.0   
224                      Yemen   3.440000e+08                      13.0   
225                     Zambia   4.000000e+08                      26.0   
226                   Zimbabwe   4.800000e+08                      32.0   

     % Renewable  
0      78.669280  
1     100.000000  
2       0.551010  
3       0.641026  
4   

Завантажуємо GDP

In [164]:
GDP = pd.read_csv("API_NY.GDP.MKTP.CD_DS2_en_csv_v2_76261.csv", skiprows=4)

Перейменовуємо країни

In [165]:
rename_gdp = {
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}

GDP['Country Name'] = GDP['Country Name'].replace(rename_gdp)

Вибираємо тільки потрібні колонки (2006-2015)

In [166]:
columns_to_keep = ['Country Name'] + [str(year) for year in range(2006, 2016)]
GDP = GDP[columns_to_keep]

Завантажуємо scimagojr

In [167]:
def answer_one():
    ScimEn= pd.read_excel('scimagojr country rank.xlsx')

    #Вибираємо топ 15 країн
    ScimEn = ScimEn[ScimEn['Rank'] <= 15]

    # Об'єднання даних (перетин країн)
    merged_df = ScimEn.merge(energy, how='inner', left_on='Country', right_on='Country')
    merged_df = merged_df.merge(GDP, how='inner', left_on='Country', right_on='Country Name')

    # Видалення дублікату колонки Country Name
    merged_df.drop(columns=['Country Name'], inplace=True)

    # Встановлення індексу
    merged_df.set_index('Country', inplace=True)

    # Вибір потрібних колонок у правильному порядку
    final_columns = [
        'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 
        'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', 
        '% Renewable'
    ] + [str(year) for year in range(2006, 2016)]
    
    final_df = merged_df[final_columns]

    return final_df

result = answer_one()

print(result)
    

                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     402188             400158    5077232   
United States          2     208042             203356    3540288   
India                  3      81853              79757     968523   
Japan                  4      58342              57735     717946   
United Kingdom         5      56288              54713    1095371   
Germany                6      50906              49773     777362   
Russian Federation     7      46186              45868     217996   
Canada                 8      41209              40390     915491   
Italy                  9      38700              36909     639473   
South Korea           10      38362              38046     618762   
Iran                  11      32080              31725     634135   
France                12      30810              30139     547122   
Spain                 13      2984

Question 2

In [168]:
def answer_two():
    Top15 = answer_one()

    # Вибираємо тільки колонки з ВВП (2006-2015)
    gdp_columns = [str(year) for year in range(2006, 2016)]

    # Обчислюємо середній ВВП для кожної країни (mean по рядках)
    avgGDP = Top15[gdp_columns].mean(axis=1)

    # Сортуємо за спаданням
    avgGDP = avgGDP.sort_values(ascending=False)

    return avgGDP
print(answer_two())

Country
United States         1.572243e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.590729e+12
United Kingdom        2.777505e+12
France                2.692000e+12
Italy                 2.152983e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.406644e+12
South Korea           1.221328e+12
Australia             1.207997e+12
Iran                  4.567516e+11
dtype: float64


Question 3

In [169]:
def answer_three():
    Top15 = answer_one()

    # Обчислюємо середній ВВП та отримуємо країну на 6-му місці
    avgGDP = Top15[[str(year) for year in range(2006, 2016)]].mean(axis=1)
    sixth_country = avgGDP.sort_values(ascending=False).index[5]

    # Обчислюємо різницю ВВП між 2015 і 2006 роками
    gdp_change = Top15.loc[sixth_country, '2015'] - Top15.loc[sixth_country, '2006']

    return gdp_change
print(answer_three())

124621907951.68018


Question 4

In [170]:
def answer_four():
    Top15 = answer_one()

    # Обчислюємо відношення Self-Citations / Citations
    Top15["Citation Ratio"] = Top15["Self-citations"] / Top15["Citations"]

    # Знаходимо країну з максимальним значенням
    max_country = Top15["Citation Ratio"].idxmax()
    max_value = Top15["Citation Ratio"].max()

    return (max_country, max_value)
print(answer_four())

('China', 0.6916510807463594)


Question 5

In [171]:
def answer_five():
    Top15 = answer_one()

    # Обчислюємо населення
    Top15["Population Estimate"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]

    # Отримуємо країну з 3-м за чисельністю населення
    third_most_populous = Top15["Population Estimate"].sort_values(ascending=False).index[2]

    return third_most_populous
print(answer_five())

United States


Question 6

In [172]:
def answer_six():
    Top15 = answer_one()

    # Оцінка населення
    Top15["Population Estimate"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]

    # Citable documents per capita
    Top15["Citable Docs per Capita"] = Top15["Citable documents"] / Top15["Population Estimate"]

    # Обчислюємо кореляцію
    correlation = Top15["Citable Docs per Capita"].corr(Top15["Energy Supply per Capita"])

    return correlation
print(answer_six())

0.7019601723071185


Question 7

In [173]:
def answer_seven():
    Top15 = answer_one()

    # Словник континентів
    ContinentDict = {
        'China': 'Asia', 
        'United States': 'North America', 
        'Japan': 'Asia', 
        'United Kingdom': 'Europe', 
        'Russian Federation': 'Europe', 
        'Canada': 'North America', 
        'Germany': 'Europe', 
        'India': 'Asia',
        'France': 'Europe', 
        'South Korea': 'Asia', 
        'Italy': 'Europe', 
        'Spain': 'Europe', 
        'Iran': 'Asia',
        'Australia': 'Australia', 
        'Brazil': 'South America'
    }

    # Створюємо стовпець 'Continent' в Top15 за допомогою словника
    Top15['Continent'] = Top15.index.map(ContinentDict)

    # Оцінка населення
    Top15["Population Estimate"] = Top15["Energy Supply"] / Top15["Energy Supply per Capita"]

    # Групуємо по континентах і обчислюємо потрібні статистики
    continent_stats = Top15.groupby('Continent')['Population Estimate'].agg(['size', 'sum', 'mean', 'std'])

    return continent_stats
print(answer_seven())

               size           sum          mean           std
Continent                                                    
Asia              5  2.898666e+09  5.797333e+08  6.790979e+08
Australia         1  2.331602e+07  2.331602e+07           NaN
Europe            6  4.579297e+08  7.632161e+07  3.464767e+07
North America     2  3.528552e+08  1.764276e+08  1.996696e+08
South America     1  2.059153e+08  2.059153e+08           NaN
